In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

In [32]:
consumption=pd.read_excel("C:\\Users\\sudhi\\OneDrive\\Desktop\\Analytics Labs\\Data Engineer\\ML and AI\\Case Study\\11. Capstone Case Study - Predict Cred Card Consumption\\CreditConsumptionData.xlsx")
behaviour =pd.read_excel("C:\\Users\\sudhi\\OneDrive\\Desktop\\Analytics Labs\\Data Engineer\\ML and AI\\Case Study\\11. Capstone Case Study - Predict Cred Card Consumption\\CustomerBehaviorData.xlsx")
demographic=pd.read_excel("C:\\Users\\sudhi\\OneDrive\\Desktop\\Analytics Labs\\Data Engineer\\ML and AI\\Case Study\\11. Capstone Case Study - Predict Cred Card Consumption\\CustomerDemographics.xlsx")

In [68]:
df=pd.merge(left=behaviour,right=consumption,left_on=consumption['ID'],right_on=behaviour['ID'],how='inner')

df=df.drop('key_0',axis=1)

df=pd.merge(left=df,right=demographic,left_on=df['ID_y'],right_on=demographic['ID'],how='inner')

df=df.drop(['key_0','ID_x','ID_y','ID'],axis=1)

###### Removed column key_0,ID_x, ID_y,ID as it is not required for prediction also  unique identifiers can be redundant 

In [69]:
df_new = df.loc[ df.cc_cons.isna() ]
df_new.drop('cc_cons',axis=1,inplace=True)
df_new.dropna(inplace=True)

In [70]:
# separate the new and existing customers
df = df.loc[ df.cc_cons.notna() ]
df.dropna(inplace=True)

In [71]:
y=df['cc_cons']

In [72]:
df.drop('cc_cons',axis=1,inplace=True)

In [73]:
df_num=df.select_dtypes(exclude ='object').columns

In [74]:
df_cat=df.select_dtypes(include ='object').columns

In [40]:
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler , OneHotEncoder ,StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [42]:
ct1 = ColumnTransformer(
    transformers=[
    ('olt',RobustScaler(),df_num),#outlier_treatment
    ('Ohe',OneHotEncoder(drop='first'),df_cat),#onehot encoding
    ('std',StandardScaler(),df_num)#standard scaler
                    ]
)

In [43]:
lr=LinearRegression()
rfr = RandomForestRegressor(max_depth=3)

In [44]:
pipe1=Pipeline(steps=[
    ('col_transformation',ct1),
    ('linear_regression',lr)
])

In [45]:
pipe2=Pipeline(steps=[
    ('col_transformation',ct1),
    ('linear_regression',rfr)
])

In [46]:
pipe1.fit(X_train,y_train)

Pipeline(steps=[('col_transformation',
                 ColumnTransformer(transformers=[('olt', RobustScaler(),
                                                  Index(['cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed...
       'max_credit_amount_apr', 'debit_amount_may', 'credit_amount_may',
       'credit_count_may', 'debit_count_may', 'max_credit_amount_may',
       'debit_amount_jun', 'credit_amount_jun', 'credit_count_jun',
       'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction'],
      dtype='object'))])),
                ('linear_regression', LinearRegression())])

In [47]:
train_preds_pipe1 = pipe1.predict(X_train)
test_preds_pipe1 = pipe1.predict(X_test)

In [86]:
print("MAE train pipe1 :: ",mean_absolute_error(train_preds_pipe1, y_train))
print("MAE test pipe1 :: ",mean_absolute_error(test_preds_pipe1, y_test))
print("MAPE train pipe1 :: ",mean_absolute_percentage_error(train_preds_pipe1, y_train))
print("MAPE test pipe1 :: ",mean_absolute_percentage_error(test_preds_pipe1, y_test))
cross_val_scores_pipe1 = cross_val_score(estimator=pipe1, X = X_train, y = y_train, cv = 3)
print("cross_val_score pipe1 ::", cross_val_scores_pipe1.mean())

MAE train pipe1 ::  70959.91114360378
MAE test pipe1 ::  69143.38247625783
MAPE train pipe1 ::  1.263101893655869
MAPE test pipe1 ::  1.2429670131273414
cross_val_score pipe1 :: -0.005741467424490665


In [49]:
pipe2.fit(X_train,y_train)

Pipeline(steps=[('col_transformation',
                 ColumnTransformer(transformers=[('olt', RobustScaler(),
                                                  Index(['cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed...
       'credit_count_may', 'debit_count_may', 'max_credit_amount_may',
       'debit_amount_jun', 'credit_amount_jun', 'credit_count_jun',
       'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction'],
      dtype='object'))])),
                ('linear_regression', RandomForestRegressor(max_depth=3))])

In [50]:
train_preds_pipe2 = pipe2.predict(X_train)
test_preds_pipe2 = pipe2.predict(X_test)

In [87]:
print("MAE train pipe2 :: ",mean_absolute_error(train_preds_pipe2, y_train))
print("MAE test  pipe2 :: ",mean_absolute_error(test_preds_pipe2, y_test))
print("MAPE train  pipe2 :: ",mean_absolute_percentage_error(train_preds_pipe2, y_train))
print("MAPE test  pipe2 :: ",mean_absolute_percentage_error(test_preds_pipe2, y_test))
cross_val_scores_pipe2 = cross_val_score(estimator=pipe2, X = X_train, y = y_train, cv = 3)
print("cross_val_score pipe2 ::", cross_val_scores_pipe2.mean())

MAE train pipe2 ::  70841.16720151236
MAE test  pipe2 ::  69239.99390178561
MAPE train  pipe2 ::  1.2480233203120947
MAPE test  pipe2 ::  1.2290608468996638
cross_val_score pipe2 :: -0.0016119642884542262


In [82]:
pred=pd.DataFrame(pipe2.predict(df_new))

In [83]:
df_new=df_new.reset_index(drop = True)

In [84]:
df_new['pred_cc_cons']=pred

In [85]:
df_new

,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,...,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction,pred_cc_cons
0,2795.450,1290.00,6141.05,676.50,9146.10,15479.00,4.0,10,4,47,...,current,M,37,MEDIUM,11.9,4,575.0,1,16.0,55574.975481
1,29405.130,1640.00,1670.00,2463.92,10947.50,956.00,3.0,10,4,50,...,current,M,33,MEDIUM,7.8,6,394.0,0,19.0,55382.848915
2,5985.200,6189.00,2696.09,766.00,716.00,4133.22,7.0,30,5,24,...,current,M,53,LOW,33.0,10,324.0,1,15.0,55235.051222
3,2105.930,18225.00,34763.56,167.00,4260.27,20185.00,1.0,99,71,5,...,current,M,33,MEDIUM,7.8,5,370.0,1,7.0,62368.249634
4,3269.000,3532.00,3158.40,2699.77,3373.48,5120.00,2.0,10,47,5,...,current,M,62,LOW,12.6,9,505.0,1,3.0,63959.280071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988,3043.000,742.20,3418.00,2181.94,12032.60,788.00,1.0,5,6,1,...,current,M,60,MEDIUM,24.0,5,968.0,0,9.0,55835.693475
4989,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,4,1,...,current,M,35,MEDIUM,13.5,1,523.0,0,7.0,56137.683817
4990,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,50,18,...,current,M,53,MEDIUM,9.9,7,723.0,1,2.0,57710.852980
4991,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,1,1,...,current,M,62,HIGH,37.8,8,863.0,1,16.0,56418.009486
